In [21]:
import pickle
from flask import Flask, request, app,jsonify,url_for,render_template
import numpy as np
import pandas as pd
from custom_encoder import GetDummies


In [22]:
## Load the model
grid_model_rfc = pickle.load(open("models/grid_model_rfc.pkl","rb"))
sc = pickle.load(open("models/credit_score_multi_class_sc.pkl","rb"))
ohe = pickle.load(open("models/credit_score_multi_class_ohe_encoder.pkl","rb"))
le = pickle.load(open("models/credit_score_multi_class_le.pkl","rb"))

In [23]:
dummy = pickle.load(open("models/credit_score_multi_class_get_dummy.pkl", "rb"))

In [24]:
df = pd.read_csv("Data/cleaned_test.csv")
for i in df.columns:
    if i in ["Customer_ID", "Month"]:
        df.drop(columns=[i], inplace=True)

In [25]:
X_test = df.copy()

In [26]:
X_test_dummy = dummy.transform(X_test)
X_test_dummy = X_test_dummy.reset_index(drop=True)

In [27]:
X_test_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18744 entries, 0 to 18743
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          18744 non-null  float64
 1   Occupation                   18744 non-null  object 
 2   Annual_Income                18744 non-null  float64
 3   Monthly_Inhand_Salary        18744 non-null  float64
 4   Num_Bank_Accounts            18744 non-null  int64  
 5   Num_Credit_Card              18744 non-null  int64  
 6   Interest_Rate                18744 non-null  int64  
 7   Num_of_Loan                  18744 non-null  float64
 8   Delay_from_due_date          18744 non-null  float64
 9   Num_of_Delayed_Payment       18744 non-null  float64
 10  Changed_Credit_Limit         18744 non-null  float64
 11  Num_Credit_Inquiries         18744 non-null  float64
 12  Credit_Mix                   18744 non-null  object 
 13  Outstanding_Debt

In [28]:
X_test_dummy.columns

Index(['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'ToL_No Data', 'ToL_auto loan', 'ToL_credit-builder loan',
       'ToL_debt consolidation loan', 'ToL_home equity loan',
       'ToL_mortgage loan', 'ToL_not specified', 'ToL_payday loan',
       'ToL_personal loan', 'ToL_student loan'],
      dtype='object')

In [29]:
X_test.columns

Index(['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance'],
      dtype='object')

In [30]:
cat = X_test_dummy.select_dtypes(include="object").columns.tolist()

X_test_cat = pd.DataFrame(
    ohe.transform(X_test[cat]), columns=ohe.get_feature_names_out()
)
X_test_ohe = X_test_cat.join(X_test_dummy.select_dtypes("number"))

In [31]:
X_test_scaled = sc.transform(X_test_ohe)

In [32]:
y_pred = grid_model_rfc.predict(X_test_scaled)

In [33]:
y_pred = le.inverse_transform(y_pred)

In [34]:
df.shape

(18744, 22)

In [35]:
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ["Prediction_Credit_Score"]

In [36]:
y_pred

,Prediction_Credit_Score
0,Good
1,Good
2,Good
3,Good
4,Good
...,...
18739,Good
18740,Good
18741,Good
18742,Poor


In [37]:
test_df = pd.read_csv("Data/cleaned_test.csv")

In [38]:
temp = pd.concat([df,y_pred],axis=1)

In [39]:
temp.columns

Index(['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Prediction_Credit_Score'],
      dtype='object')

In [40]:
temp

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Prediction_Credit_Score
0,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"auto loan,credit-builder loan,personal loan,ho...",3.0,...,Good,809.98,35.030402,273.0,No,49.574949,236.642682,Low_spent_Small_value_payments,186.266702,Good
1,24.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"auto loan,credit-builder loan,personal loan,ho...",3.0,...,Good,809.98,33.053114,274.0,No,49.574949,21.465380,High_spent_Medium_value_payments,361.444004,Good
2,28.0,Teacher,34847.84,3037.986667,2,4,6,1.0,credit-builder loan,3.0,...,Good,605.03,30.116600,328.0,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083,Good
3,35.0,Engineer,143162.64,12187.220000,1,5,8,3.0,"auto loan,auto loan,not specified",6.0,...,Good,1303.01,35.685836,222.0,No,246.992319,453.615131,Low_spent_Large_value_payments,788.114550,Good
4,35.0,Engineer,143162.64,12187.220000,1,5,8,1381.0,"auto loan,auto loan,not specified",8.0,...,Good,1303.01,31.819845,223.0,No,246.992319,841.232236,Low_spent_Medium_value_payments,410.497445,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18739,45.0,Media_Manager,16680.35,1528.029167,1,1,5,4.0,"payday loan,student loan,mortgage loan,not spe...",1.0,...,Good,897.16,39.153349,261.0,No,41.113561,101.813161,Low_spent_Medium_value_payments,289.876195,Good
18740,50.0,Writer,37188.10,3097.008333,1,4,5261,3.0,"home equity loan,mortgage loan,student loan",7.0,...,Good,620.64,32.740865,365.0,No,84.205949,165.115322,Low_spent_Small_value_payments,350.379562,Good
18741,50.0,Writer,37188.10,3097.008333,1,4,4252,3.0,"home equity loan,mortgage loan,student loan",7.0,...,Good,620.64,25.708414,367.0,No,84.205949,183.365628,Low_spent_Large_value_payments,312.129256,Good
18742,29.0,Architect,20002.88,1929.906667,10,8,29,5.0,"personal loan,auto loan,mortgage loan,student ...",33.0,...,Bad,3571.70,32.391288,76.0,Yes,60.964772,107.210742,Low_spent_Small_value_payments,314.815153,Poor


In [42]:
temp = pd.concat([test_df,y_pred],axis=1)

In [43]:
temp["Prediction_Credit_Score"].value_counts()

Prediction_Credit_Score
Poor        7404
Standard    6371
Good        4969
Name: count, dtype: int64

In [44]:
pd.read_csv("Result/prediction_on_test_data.csv")

,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Prediction_Credit_Score
0,CUS_0xd40,9,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,...,Good,809.98,35.030402,273.0,No,49.574949,236.642682,Low_spent_Small_value_payments,186.266702,Good
1,CUS_0xd40,10,24.0,Scientist,19114.12,1824.843333,3,4,3,4.0,...,Good,809.98,33.053114,274.0,No,49.574949,21.465380,High_spent_Medium_value_payments,361.444004,Good
2,CUS_0x21b1,10,28.0,Teacher,34847.84,3037.986667,2,4,6,1.0,...,Good,605.03,30.116600,328.0,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083,Good
3,CUS_0x2dbc,10,35.0,Engineer,143162.64,12187.220000,1,5,8,3.0,...,Good,1303.01,35.685836,222.0,No,246.992319,453.615131,Low_spent_Large_value_payments,788.114550,Good
4,CUS_0x2dbc,11,35.0,Engineer,143162.64,12187.220000,1,5,8,1381.0,...,Good,1303.01,31.819845,223.0,No,246.992319,841.232236,Low_spent_Medium_value_payments,410.497445,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18739,CUS_0xf16,9,45.0,Media_Manager,16680.35,1528.029167,1,1,5,4.0,...,Good,897.16,39.153349,261.0,No,41.113561,101.813161,Low_spent_Medium_value_payments,289.876195,Good
18740,CUS_0xaf61,9,50.0,Writer,37188.10,3097.008333,1,4,5261,3.0,...,Good,620.64,32.740865,365.0,No,84.205949,165.115322,Low_spent_Small_value_payments,350.379562,Good
18741,CUS_0xaf61,11,50.0,Writer,37188.10,3097.008333,1,4,4252,3.0,...,Good,620.64,25.708414,367.0,No,84.205949,183.365628,Low_spent_Large_value_payments,312.129256,Good
18742,CUS_0x8600,9,29.0,Architect,20002.88,1929.906667,10,8,29,5.0,...,Bad,3571.70,32.391288,76.0,Yes,60.964772,107.210742,Low_spent_Small_value_payments,314.815153,Poor
